In [ ]:
# default_exp utils

# Utilities

In [ ]:
#hide
from fastcore.test import *
from nbdev.showdoc import *

In [ ]:
#export

from pathlib import Path
from fastcore.basics import patch

In [ ]:
#export

@patch
def ls_sorted(self:Path):
    "ls but sorts files by name numerically"
    return self.ls().sorted(key=lambda f: int(f.with_suffix('').name))

In [ ]:
#export

def flatten_dict(d: dict):
    """flattens a nested dict one level"""
    def func(dct):
        for k, v in dct.items():
            if isinstance(v, dict):
                yield from v.items()
            else:
                yield k, v
    return dict(func(d))

In [ ]:
#hide
actual = flatten_dict(dict(a=1,b=2,c=dict(d=3,e=4),f=dict(g=dict(h=5,i=6))))
test_eq(actual, {'a': 1, 'b': 2, 'd': 3, 'e': 4, 'g': {'h': 5, 'i': 6}})

In [ ]:
#export
from collections import Counter

def most_common(lst):
    return Counter(lst).most_common(1)[0][0]

In [ ]:
#hide
test_eq(most_common([1,1,1,2,2,3,3,3,3,4,4]), 3)
test_eq(most_common([1,1,1,2,2,3,3,3,4,4]), 1)
test_eq(most_common([0]), 0)

In [ ]:
#export

def get_node(tree: dict, path: str, sep: str = '.'):
    if path is None or path == '':
        return tree
    fields = path.split(sep)
    node = tree
    for field in fields:
        node = node[field]
    return node


In [ ]:
#hide
tree = {
    'a': 1,
    'b': {
        'c': {
            'd': 2,
        },
        'e': 3,
    },
    'f': 4,
}

test_eq(get_node(tree, 'a'), 1)
test_eq(get_node(tree, 'b.c.d'), 2)
test_eq(get_node(tree, 'b.e'), 3)
test_eq(get_node(tree, 'b/e', sep='/'), 3)
test_eq(get_node(tree, None), tree)

In [ ]:
#export
def apply_nested(tree: dict, path: str, func, sep: str = '.'):
    parts = path.split(sep)
    parent_node = get_node(tree, sep.join(parts[:-1]))
    parent_node[parts[-1]] = func(parent_node[parts[-1]])
    return tree

In [ ]:
#hide
tree = {
    'a': 5,
    'b': {
        'c': {
            'd': 2,
        },
        'e': 3,
    },
    'f': 4,
}

func = lambda x: x*x

for path in ['a', 'b.c.d', 'f']:
    apply_nested(tree, path, func)

test_eq(tree['a'], 25)
test_eq(tree['b']['c']['d'], 4)
test_eq(tree['b']['e'], 3)
test_eq(tree['f'], 16)

In [ ]:
#export
from pathlib import Path

def resolve_path(config, field_path, sep='.'):
    func = lambda s: str(Path(s).resolve())
    return apply_nested(config, field_path, func, sep)

In [ ]:
#hide
config = {
    'data_path': './a/b/c',
    'model': {
        'save_path': './path/to/artifact'
    }
}

resolve_path(config, 'data_path')
resolve_path(config, 'model.save_path')

assert not config['data_path'].startswith('.')
assert not config['model']['save_path'].startswith('.')